In [26]:
from transformers import pipeline
from PIL import Image, ImageChops
import requests
import numpy as np
import scipy
import cv2

IMG_PATH = "/Users/herk/Downloads/skydive.jpg"

In [27]:
# load pipe
pipe = pipeline(task="depth-estimation", model="LiheYoung/depth-anything-large-hf")

# load image
image = Image.open(IMG_PATH)

# inference
depth = pipe(image)["depth"]
depth.save("/Users/herk/Downloads/depth.jpg")

In [28]:
def shift_image(img, depth_img, shift_amount=10):
    # Ensure base image has alpha
    img = img.convert("RGBA")
    data = np.array(img)

    # Ensure depth image is grayscale (for single value)
    depth_img = depth_img.convert("L")
    depth_data = np.array(depth_img)
    deltas = np.array((depth_data / 255.0) * float(shift_amount), dtype=int)

    # This creates the transprent resulting image.
    # For now, we're dealing with pixel data.
    shifted_data = np.zeros_like(data)

    width = img.width
    height = img.height

    for y, row in enumerate(deltas):
        width = len(row)
        x = 0
        while x < width:
            dx = row[x]
            if x+dx >= width:
                break
            if x-dx < 0:
                shifted_data[y][x-dx] = [0,0,0,0]
            else:
                shifted_data[y][x-dx] = data[y][x]
            x += 1

    # Convert the pixel data to an image.
    shifted_image = Image.fromarray(shifted_data)

    alphas_image = Image.fromarray(scipy.ndimage.binary_fill_holes(
      ImageChops.invert(
          shifted_image.getchannel("A")
      ))).convert("1")
    shifted_image.putalpha(ImageChops.invert(alphas_image))

    return shifted_image

In [29]:
def shift_and_inpaint(path, amount):
    shifted = shift_image(image, depth, shift_amount=amount).save(path)
    org_img = cv2.imread(filename=path)
    damaged_img = cv2.imread(filename=path)
     
    # get the shape of the image
    height, width = damaged_img.shape[0], damaged_img.shape[1]
     
    # Converting all pixels greater than zero to black while black becomes white
    for i in range(height):
        for j in range(width):
            if damaged_img[i, j].sum() > 0:
                damaged_img[i, j] = 0
            else:
                damaged_img[i, j] = [255, 255, 255]
     
    # saving the mask 
    mask = cv2.cvtColor(damaged_img, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("title", mask)
    #cv2.waitKey(0)

    dst = cv2.inpaint(org_img, mask, 3, cv2.INPAINT_NS)
    # Write the output.
    cv2.imwrite(path, dst)

In [32]:
shift_and_inpaint("/Users/herk/Downloads/cart_left.png", 10)
shift_and_inpaint("/Users/herk/Downloads/cart_right.png", 50)

In [50]:

from PIL import Image
import pillow_heif
import math

def load_image(path):
    try:
        return Image.open(path)
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

def combine_left_and_right():
    left_img_path = "/Users/herk/Downloads/cart_left.png"
    right_img_path = "/Users/herk/Downloads/cart_right.png"
    new_image_path = "/Users/herk/Downloads/cart_with_depth.heic"

    left_img = load_image(left_img_path)
    right_img = load_image(right_img_path)

    print(f"left_img: {left_img}")
    print(f"right_img: {right_img}")

    if left_img is None or right_img is None:
        print("One of the images could not be loaded.")
        return
    
    image_width = left_img.size[0]
    image_height = left_img.size[1]
    fov_horizontal_degrees = 55
    fov_horizontal_radians = fov_horizontal_degrees * (3.14159 / 180)
    focal_length_pixels = 0.5 * image_width / math.tan(0.5 * fov_horizontal_radians)
    baseline = 65.0 # in millimeters

    # Since pillow_heif does not directly support adding depth information,
    # the following is an approximation that creates a HEIF image with two pictures.
    # Depth information and camera intrinsics are not standard and require
    # specific API support which is not available in pillow_heif as of the last knowledge update.

    left_img_bytes = left_img.tobytes("raw")
    right_img_bytes = right_img.tobytes("raw")

    cv_img = cv2.imread(left_img_path, cv2.IMREAD_UNCHANGED)
    heif_file_1 = pillow_heif.from_bytes(
        mode="BGRA;16",
        size=(cv_img.shape[1], cv_img.shape[0]),
        data=bytes(cv_img)
    )
    cv_img = cv2.imread(right_img_path, cv2.IMREAD_UNCHANGED)
    heif_file_2 = pillow_heif.from_bytes(
        mode="BGRA;16",
        size=(cv_img.shape[1], cv_img.shape[0]),
        data=bytes(cv_img)
    )
    heif_file_1.add_from_bytes( mode="BGRA;16",
        size=(cv_img.shape[1], cv_img.shape[0]),
        data=bytes(cv_img))
    heif_file.write_to_file(new_image_path)

combine_left_and_right()

left_img: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=669x446 at 0x29A656C50>
right_img: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=669x446 at 0x29A655B70>


AttributeError: 'HeifFile' object has no attribute 'add_from_bytes'

In [39]:
read = pillow_heif.read_heif("/Users/herk/Downloads/cart_with_depth.heic")

<HeifFile with 2 images: ['<HeifImage 669x446 RGB with 895122 bytes image data and 0 thumbnails>', '<HeifImage 669x446 RGB with 895122 bytes image data and 0 thumbnails>']>

In [44]:
pillow_heif.HeifFile()

<HeifFile with 0 images: []>